In [1]:
import pandas as pd

/var/folders/33/b1s2x_2166q65n2gxsrf_h9h0000gn/T/ipykernel_22847/4080736814.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


## Read all the data

In [2]:
left=pd.read_csv('karolinska.csv')
pmcoaki=pd.read_csv('./pmcoalist_ki.csv')
pmcoaki = pmcoaki.rename(columns={'PMID':'pmid'})

left=left.dropna(subset=['is_open_code','is_open_data','is_register_pred','is_coi_pred','is_fund_pred'])
left=pd.merge(left,pmcoaki,on='pmid')
left=left[left.is_research_x]
columns=['pmid','is_coi_pred','is_register_pred','is_fund_pred','is_open_code','is_open_data','Publication Year','Institution','Journal/Book']
left=pd.DataFrame(left, columns=columns)

In [3]:
right=pd.read_csv('kidept_pmids_orgbased_2010_2022.csv')

In [4]:
df=left.set_index('pmid').join(right.set_index('pmid'), how='inner')
df['is_open_code']=df['is_open_code'].astype('bool')
df['is_open_data']=df['is_open_data'].astype('bool')

In [5]:
df=df[df['Publication Year'].isin([2017,2018,2019,2020,2021])]

In [6]:
df.to_csv('raw.csv')

## Remove institutions with too few publications

In [19]:
remove_low=pd.DataFrame(df.groupby([ "Publication Year", "short_name"] ).size().reset_index())
indices=remove_low[0][lambda x: x>50].index.tolist()
remove_low=remove_low.iloc[indices]
inst=set(remove_low.short_name)
inst=list(inst)

df_clean=df[df['short_name'].isin(inst)]

In [20]:
round(df_clean.groupby(['Publication Year','short_name']).mean()*100,1).to_csv('general.csv')

# Map to specific groupings

## SOLVE THIS WHEN YOU GET BACK

In [21]:
kinord=['GPH','CNS', 'KI DS','KBH','LIME','LIME', 'MedS','MMK','OnkPat']
kisolna=['CMB','FyFa','MBB','MEB','MTC','IMM','Neuro','NVS','DentMed']
kisyd=['BioNut','KI SÖS','CLINTEC','LabMed','MedH']
def categorise(row):  
    i=row['short_name']
    if any(i in kinord for i in kinord):
        return 'KI Nord'
    elif any(i in kisyd for i in kisyd):
        return 'KI Syd'
    elif any(i in kisolna for i in kisolna):
        return 'KI Solna'


In [22]:
ki_nord=df_clean[df_clean.short_name.isin(['GPH','CNS', 'KI DS','KBH','LIME','LIME', 'MedS','MMK','OnkPat'])]
ki_syd=df_clean[df_clean.short_name.isin(['BioNut','KI SÖS','CLINTEC','LabMed','MedH'])]
ki_solna=df_clean[df_clean.short_name.isin(['CMB','FyFa','MBB','MEB','MTC','IMM','Neuro','NVS','DentMed'])]


In [23]:
ki_nord['short_name']='KI Nord'
ki_solna['short_name']='KI Solna'
ki_syd['short_name']='KI Syd'

/var/folders/wx/sx67klw51p99ypb39mtcwx_00000gn/T/ipykernel_4435/2803223710.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ki_nord['short_name']='KI Nord'
/var/folders/wx/sx67klw51p99ypb39mtcwx_00000gn/T/ipykernel_4435/2803223710.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ki_solna['short_name']='KI Solna'
/var/folders/wx/sx67klw51p99ypb39mtcwx_00000gn/T/ipykernel_4435/2803223710.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_

In [27]:
pd.options.display.max_rows = 4000
round(pd.concat([ki_nord, ki_syd,ki_solna,df_clean]).groupby(['Publication Year','short_name']).mean()*100,1).to_csv('groups_and_institutions.csv')

In [25]:
round(df_clean.groupby(['Publication Year','short_name']).mean()*100,1).to_csv('general.csv')

In [5]:
df=df[df['short_name']=='CNS']

In [6]:
df.to_csv('cns.csv')

In [26]:
round(pd.concat([ki_nord, ki_syd,ki_solna,ki_cns]).groupby(['Publication Year','Group']).mean()*100,1).to_csv('cns_pres_data.csv')

NameError: name 'ki_cns' is not defined

In [ ]:
df2 = pd.concat([ki_nord, ki_syd,ki_solna,ki_cns]).groupby(['Publication Year','Group'])['Group'].count()

In [ ]:
ki_nord.groupby('Group')['Journal/Book'].value_counts()

Group    Journal/Book              
KI Nord  PLoS One                      447
         Sci Rep                       378
         BMJ Open                      320
         Nat Commun                    145
         BMC Public Health             120
                                      ... 
         World J Emerg Surg              1
         Worldviews Evid Based Nurs      1
         Zoonoses Public Health          1
         eNeuro                          1
         mBio                            1
Name: Journal/Book, Length: 932, dtype: int64

In [ ]:
ki_solna.groupby('Group')['Journal/Book'].value_counts()

Group     Journal/Book              
KI Solna  Sci Rep                       330
          PLoS One                      252
          Nat Commun                    247
          BMJ Open                      155
          Elife                          63
                                       ... 
          Womens Health (Lond)            1
          World J Biol Psychiatry         1
          World J Surg Oncol              1
          Worldviews Evid Based Nurs      1
          Z Gerontol Geriatr              1
Name: Journal/Book, Length: 777, dtype: int64

In [ ]:
ki_syd.groupby('Group')['Journal/Book'].value_counts()

Group   Journal/Book         
KI Syd  Sci Rep                  166
        PLoS One                 125
        Nat Commun                76
        Front Immunol             54
        BMJ Open                  51
                                ... 
        Travel Med Infect Dis      1
        Unfallchirurg              1
        Updates Surg               1
        Vox Sang                   1
        World J Surg Oncol         1
Name: Journal/Book, Length: 517, dtype: int64

In [ ]:
ki_cns.groupby('Group')['Journal/Book'].value_counts()

Group  Journal/Book                   
CNS    Sci Rep                            86
       PLoS One                           67
       BMJ Open                           62
       BMC Psychiatry                     47
       Int J Environ Res Public Health    34
                                          ..
       Violence Against Women              1
       Work                                1
       World J Surg                        1
       Worldviews Evid Based Nurs          1
       eNeuro                              1
Name: Journal/Book, Length: 364, dtype: int64

In [ ]:
pd.options.display.max_seq_items = 2000

pd.concat([ki_nord, ki_syd,ki_solna,ki_cns]).groupby('Group')['Journal/Book'].value_counts()

Group   Journal/Book                   
CNS     Sci Rep                            86
        PLoS One                           67
        BMJ Open                           62
        BMC Psychiatry                     47
        Int J Environ Res Public Health    34
                                           ..
KI Syd  Travel Med Infect Dis               1
        Unfallchirurg                       1
        Updates Surg                        1
        Vox Sang                            1
        World J Surg Oncol                  1
Name: Journal/Book, Length: 2590, dtype: int64

In [ ]:
df2.to_csv('publications_cns_pres.csv')

In [ ]:
round(ki_nord.groupby(['Publication Year','short_name']).mean()*100,1).to_csv('ki_nord.csv')
round(ki_syd.groupby(['Publication Year','short_name']).mean()*100,1).to_csv('ki_syd.csv')
round(ki_solna.groupby(['Publication Year','short_name']).mean()*100,1).to_csv('ki_solna.csv')